In [14]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os

# 函數：獲取納斯達克 100 成份股的 ticker 列表
def get_nasdaq100_tickers():
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[1].text.strip()
        tickers.append(ticker)
    return tickers

# 函數：計算單一股票的 P/S ratio
def get_ps_ratio(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        revenue = info.get('totalRevenue', None)
        
        if market_cap and revenue and revenue > 0:
            ps_ratio = market_cap / revenue
            return ps_ratio
        else:
            return None
    except Exception as e:
        print(f"無法獲取 {ticker} 的數據: {e}")
        return None

# 主程式
def main_ndq():
    # 獲取納斯達克 100 成份股
    print("正在獲取納斯達克 100 成份股列表...")
    tickers = get_nasdaq100_tickers()
    print(f"找到 {len(tickers)} 個成份股")

    # 儲存數據的字典
    data = {'Ticker': [], 'P/S Ratio': []}

    # 遍歷每個 ticker 並計算 P/S ratio
    for ticker in tickers:
        print(f"正在處理 {ticker}...")
        ps_ratio = get_ps_ratio(ticker)
        data['Ticker'].append(ticker)
        data['P/S Ratio'].append(ps_ratio if ps_ratio is not None else float('nan'))

    # 轉換為 DataFrame
    df = pd.DataFrame(data)

    # 將結果保存到 Excel
    with pd.ExcelWriter('nasdaq100_ps_ratio.xlsx') as writer:
        df.to_excel(writer, sheet_name='詳細數據', index=False)
        if not valid_ps_ratios.empty:
            stats_df.to_excel(writer, sheet_name='統計分析', index=False)
    
    print("結果已保存至 'nasdaq100_ps_ratio.xlsx'")
    
    

In [15]:
# 函數：獲取 S&P 500 成份股的 ticker 列表
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:  # 跳過表頭
        ticker = row.find_all('td')[0].text.strip()  # 第1列是 ticker
        tickers.append(ticker.replace('.', '-'))  # 將點號替換為減號以符合 yfinance 格式
    return tickers

# 函數：計算單一股票的 P/S ratio
def get_ps_ratio(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        revenue = info.get('totalRevenue', None)
        
        if market_cap and revenue and revenue > 0:
            ps_ratio = market_cap / revenue
            return ps_ratio
        else:
            return None
    except Exception as e:
        print(f"無法獲取 {ticker} 的數據: {e}")
        return None

# 主程式
def main_sp500():
    # 獲取 S&P 500 成份股
    print("正在獲取 S&P 500 成份股列表...")
    tickers = get_sp500_tickers()
    print(f"找到 {len(tickers)} 個成份股")

    # 儲存數據的字典
    data = {'Ticker': [], 'P/S Ratio': []}

    # 遍歷每個 ticker 並計算 P/S ratio
    for ticker in tickers:
        print(f"正在處理 {ticker}...")
        ps_ratio = get_ps_ratio(ticker)
        data['Ticker'].append(ticker)
        data['P/S Ratio'].append(ps_ratio if ps_ratio is not None else float('nan'))

    # 轉換為 DataFrame
    df = pd.DataFrame(data)

    # 將結果保存到 Excel
    with pd.ExcelWriter('sp500_ps_ratio.xlsx') as writer:
        df.to_excel(writer, sheet_name='詳細數據', index=False)
        if not valid_ps_ratios.empty:
            stats_df.to_excel(writer, sheet_name='統計分析', index=False)
    
    print("結果已保存至 'sp500_ps_ratio.xlsx'")
    

In [16]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os

# 函數：獲取 S&P 500 成份股的 ticker 列表
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker.replace('.', '-'))
    return tickers

# 函數：獲取 Nasdaq 100 成份股的 ticker 列表
def get_nasdaq100_tickers():
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[1].text.strip()
        tickers.append(ticker)
    return tickers

# 函數：獲取 HACK ETF 成份股的 ticker 列表
def get_hack_tickers():
    # 備用方案：使用硬編碼列表，因為 ETF.com 抓取可能不穩定
    print("備註：HACK ETF 成份股列表為硬編碼，建議從官方來源更新")
    return ['CRWD', 'CSCO', 'PANW', 'FTNT', 'ZS', 'CHKP', 'GEN', 'OKTA', 'S', 'BAH']

# 函數：計算單一股票的 P/S ratio
def get_ps_ratio(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        market_cap = info.get('marketCap', None)
        revenue = info.get('totalRevenue', None)
        
        if market_cap and revenue and revenue > 0:
            ps_ratio = market_cap / revenue
            return ps_ratio
        else:
            return None
    except Exception as e:
        print(f"無法獲取 {ticker} 的數據: {e}")
        return None

# 函數：重新計算並保存數據
def calculate_and_save_data(index_type):
    if index_type == 'sp500':
        print("正在獲取 S&P 500 成份股列表...")
        tickers = get_sp500_tickers()
        excel_file = 'sp500_ps_ratio.xlsx'
        plot_title = 'S&P 500 P/S Ratio 分佈圖'
        plot_file = 'sp500_ps_ratio_distribution.png'
    elif index_type == 'nasdaq100':
        print("正在獲取 Nasdaq 100 成份股列表...")
        tickers = get_nasdaq100_tickers()
        excel_file = 'nasdaq100_ps_ratio.xlsx'
        plot_title = 'Nasdaq 100 P/S Ratio 分佈圖'
        plot_file = 'nasdaq100_ps_ratio_distribution.png'
    elif index_type == 'hack':
        print("正在獲取 HACK ETF 成份股列表...")
        tickers = get_hack_tickers()
        excel_file = 'hack_ps_ratio.xlsx'
        plot_title = 'HACK ETF P/S Ratio 分佈圖'
        plot_file = 'hack_ps_ratio_distribution.png'
    else:
        raise ValueError("無效的指數類型，必須是 'sp500'、'nasdaq100' 或 'hack'")

    print(f"找到 {len(tickers)} 個成份股")
    data = {'Ticker': [], 'P/S Ratio': []}
    for ticker in tickers:
        print(f"正在處理 {ticker}...")
        ps_ratio = get_ps_ratio(ticker)
        data['Ticker'].append(ticker)
        data['P/S Ratio'].append(ps_ratio if ps_ratio is not None else float('nan'))

    df = pd.DataFrame(data)
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='詳細數據', index=False)
    print(f"數據已保存至 '{excel_file}'")
    return df, excel_file, plot_title, plot_file

# 函數：分析數據並生成統計和圖表
def analyze_data(df, excel_file, plot_title, plot_file):
    valid_ps_ratios = df['P/S Ratio'].dropna()
    if not valid_ps_ratios.empty:
        mean_ps = valid_ps_ratios.mean()
        median_ps = valid_ps_ratios.median()
        std_ps = valid_ps_ratios.std()
        min_ps = valid_ps_ratios.min()
        max_ps = valid_ps_ratios.max()

        print(f"\n統計結果：")
        print(f"平均 P/S Ratio: {mean_ps:.2f}")
        print(f"中位數 P/S Ratio: {median_ps:.2f}")
        print(f"標準差: {std_ps:.2f}")
        print(f"最小值: {min_ps:.2f}")
        print(f"最大值: {max_ps:.2f}")
        print(f"有效數據點數: {len(valid_ps_ratios)}")

        # 添加統計數據到 DataFrame
        stats_df = pd.DataFrame({
            '統計項目': ['平均值', '中位數', '標準差', '最小值', '最大值', '有效數據點數'],
            '值': [mean_ps, median_ps, std_ps, min_ps, max_ps, len(valid_ps_ratios)]
        })

        # 繪製分佈圖
        plt.figure(figsize=(10, 6))
        valid_ps_ratios.hist(bins=20, edgecolor='black')
        plt.title(plot_title)
        plt.xlabel('P/S Ratio')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.savefig(plot_file)
        print(f"分佈圖已保存為 '{plot_file}'")

        # 保存統計數據到 Excel，使用 openpyxl 引擎支持追加模式
        with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            stats_df.to_excel(writer, sheet_name='統計分析', index=False)
        print(f"統計分析已更新至 '{excel_file}'")
    else:
        print("無有效數據可用於分析")

# 主程式
def main_excel():
    # 讓使用者選擇指數
    index_choice = input("請選擇指數 (sp500, nasdaq100 或 hack): ").lower()
    if index_choice not in ['sp500', 'nasdaq100', 'hack']:
        print("無效選擇，請輸入 'sp500'、'nasdaq100' 或 'hack'")
        return

    excel_file = {
        'sp500': 'sp500_ps_ratio.xlsx',
        'nasdaq100': 'nasdaq100_ps_ratio.xlsx',
        'hack': 'hack_ps_ratio.xlsx'
    }[index_choice]
    plot_title = {
        'sp500': 'S&P 500 P/S Ratio Distribution',
        'nasdaq100': 'Nasdaq 100 P/S Ratio Distribution',
        'hack': 'HACK ETF P/S Ratio Distribution'
    }[index_choice]
    plot_file = {
        'sp500': 'sp500_ps_ratio_distribution.png',
        'nasdaq100': 'nasdaq100_ps_ratio_distribution.png',
        'hack': 'hack_ps_ratio_distribution.png'
    }[index_choice]

    regenerate = input("是否重新抓取數據並計算 P/S ratio？(y/n): ").lower()

    if regenerate == 'y' or not os.path.exists(excel_file):
        # 重新計算並保存數據
        df, excel_file, plot_title, plot_file = calculate_and_save_data(index_choice)
    else:
        # 從現有 Excel 文件讀取數據
        if os.path.exists(excel_file):
            print(f"從 {excel_file} 讀取數據...")
            df = pd.read_excel(excel_file, sheet_name='詳細數據')
        else:
            print(f"Excel 文件 '{excel_file}' 不存在，將重新抓取數據...")
            df, excel_file, plot_title, plot_file = calculate_and_save_data(index_choice)

    # 分析數據
    analyze_data(df, excel_file, plot_title, plot_file)


    
main_excel()

無效選擇，請輸入 'sp500'、'nasdaq100' 或 'hack'
